# Set-up the client

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "myCluster"
print(ml_client.compute.get(cluster_name))

# Data asset

### Iris dataset

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from datetime import datetime

iris_data = Data(
    path="./data",
    type=AssetTypes.MLTABLE,
    description="Fisher's iris dataset",
    name="iris",
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)

ml_client.data.create_or_update(iris_data)

In [ ]:
import mltable

data_asset = ml_client.data.get("iris", label="latest")

tbl = mltable.load(data_asset.path)

df = tbl.to_pandas_dataframe()
df.head()

### Model architecture

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from datetime import datetime

model_asset = Data(
    name="iris_model_architecture",
    path="./iris_model/iris_model.py",
    type=AssetTypes.URI_FILE,
    description="Architecture definition for Iris classifier",
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
ml_client.data.create_or_update(model_asset)

# Environments

## Train environment

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext
from datetime import datetime

env = Environment(
    name="iris_train",
    description="PyTorch CPU training environment",
    build=BuildContext(
        path = "./environments/iris_train/",
        dockerfile_path="Dockerfile"
    ),
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)

ml_client.environments.create_or_update(env)

In [ ]:
env_check = ml_client.environments.get(
    name="iris_train",
    label = "latest"
)

print(f"✅ Environment exists: {env_check.name} v{env_check.version}")
print(f"Image URI: {env_check.image}")  # Vide si build en cours

## Unit test environment

In [ ]:
!docker build -t iris-train:latest -f environments/iris_train/Dockerfile environments/iris_train
!docker build -t iris-tests:latest -f environments/iris_tests/Dockerfile environments/iris_tests

# Unit tests

In [ ]:
pass

# Components

In [ ]:
from azure.ai.ml import load_component
from datetime import datetime

components_dir = "./components/"

data_prepare_component = load_component(source=f"{components_dir}/data_prepare/data_prepare.yaml")
ml_client.components.create_or_update(
    data_prepare_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ data_prepare registred")

data_split_component = load_component(source=f"{components_dir}/data_split/data_split.yaml")
ml_client.components.create_or_update(
    data_split_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ data_split registred")

model_train_component = load_component(source=f"{components_dir}/model_train/model_train.yaml")
ml_client.components.create_or_update(
    model_train_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ model_train registred")

model_eval_component = load_component(source=f"{components_dir}/model_eval/model_eval.yaml")
ml_client.components.create_or_update(
    model_eval_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ model_eval registred")


# Pipelines as components

In [ ]:
from pipelines.evaluation import iris_evaluation_pipeline
from datetime import datetime

registered_eval_pipeline = ml_client.components.create_or_update(
    iris_evaluation_pipeline,
    version=datetime.now().strftime("%Y%m%d%H%M%S"),
)

print(f"    Name: {registered_eval_pipeline.name}")
print(f"    Version: {registered_eval_pipeline.version}")

In [ ]:
from pipelines.production import iris_production_pipeline
from datetime import datetime

registered_prod_pipeline = ml_client.components.create_or_update(
    iris_production_pipeline,
    version=datetime.now().strftime("%Y%m%d%H%M%S"),
)

print(f"    Name: {registered_prod_pipeline.name}")
print(f"    Version: {registered_prod_pipeline.version}")

# Production job

In [ ]:
from pipelines.production import iris_production_pipeline
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

data_asset = ml_client.data.get("iris", label="latest")
model_asset = ml_client.data.get("iris_model_architecture", label="latest")

data_input = Input(
    type=AssetTypes.MLTABLE,
    path=data_asset.id,
)

model_input = Input(
    type=AssetTypes.URI_FILE,
    path=model_asset.id,
)

production_job = iris_production_pipeline(
    data=data_input,
    archi=model_input,
    compute="myCluster",
)

submitted_job = ml_client.jobs.create_or_update(
    production_job,
    experiment_name="iris-classification"
)

print(f"✅ Production pipeline submitted!")
print(f"   Job name: {submitted_job.name}")
print(f"   Studio URL: {submitted_job.studio_url}")

# Optionnel : real time job monitoring
# ml_client.jobs.stream(submitted_job.name)

## Recurrent execution

In [ ]:
# Get the job to schedule

all_jobs = ml_client.jobs.list()
jobs_list = [
    job for job in ml_client.jobs.list() 
    if job.experiment_name == "iris-classification" 
    and job.properties.get("runSource") != "Schedule"
]
job = jobs_list[0]
job

In [ ]:
from azure.ai.ml.entities import JobSchedule, RecurrenceTrigger

job =  ml_client.jobs.get(name = "frank_airport_nl42g9wj33")

schedule = JobSchedule(
    name="iris-daily-retrain",
    trigger=RecurrenceTrigger(
        frequency="hour",  # or "day", "week", "month"
        interval=1
    ),
    create_job=job.name
)

ml_client.schedules.begin_create_or_update(schedule=schedule)

## Custom parameters

In [ ]:
all_jobs = ml_client.jobs.list()
jobs_list = [
    job for job in ml_client.jobs.list() 
    if job.experiment_name == "iris-classification" 
    and job.properties.get("runSource") != "Schedule"
]
job = jobs_list[0]
job

In [ ]:
from azure.ai.ml import Input
from datetime import datetime
from azure.ai.ml.constants import AssetTypes

job =  ml_client.jobs.get(name = "frank_airport_nl42g9wj33")

production_pipeline = ml_client.components.get(
    name="iris_production_pipeline",
    label="latest",
)

custom_job = ml_client.jobs.create_or_update(
    production_pipeline(
        data=Input(
            type=AssetTypes.MLTABLE,
            path=job.inputs.data.path
        ),
        archi=Input(
            type=AssetTypes.URI_FILE,
            path=job.inputs.archi.path
        ),
        compute="myCluster",
        lr=0.001,
        epochs=20,
    ),
    experiment_name=job.experiment_name,
    display_name = "20epochs_" + datetime.now().strftime("%Y%m%d%H%M%S")
)


# Deploy the model as a real-time endpoint

### Collect the model

In [ ]:
all_jobs = ml_client.jobs.list()
jobs_list = [
    job.name for job in ml_client.jobs.list() 
    if job.experiment_name == "iris-classification" 
    and job.properties.get("runSource") != "Schedule"
]
jobs_list

In [ ]:
all_models = ml_client.models.list(name = "iris-model")
models_list = [model for model in all_models]
models_list

In [ ]:
model = models_list[1]
job_child_name = model.properties.get('azureml.artifactPrefix').split('/')[1][5:]
job_child = ml_client.jobs.get(name=job_child_name)
job_child.tags["azureml.pipeline"]

In [ ]:
print(f"Model: {model.name} - version: {model.version}")

### Create RTI service

In [ ]:
environment = ml_client.environments.get(name="iris_train", label="latest")

In [ ]:
computes = ml_client.compute.list()
for compute in computes:
    print(f"Nom: {compute.name}, Type: {compute.type}")

In [ ]:
try:
    compute = ml_client.compute.get("mykubefr")
    print(f"Compute: {compute.name}")
    print(f"Type: {compute.type}")
    print(f"Resource ID: {compute.resource_id if hasattr(compute, 'resource_id') else 'N/A'}")
    print(f"État: {compute.provisioning_state if hasattr(compute, 'provisioning_state') else 'N/A'}")
except Exception as e:
    print(f"Erreur lors de la récupération: {e}")

In [ ]:
#from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.entities import KubernetesOnlineEndpoint, KubernetesOnlineDeployment, CodeConfiguration

# endpoint
endpoint = KubernetesOnlineEndpoint(
    name="iris-endpoint",
    compute="mykubefr",
    auth_mode="key"
)
ml_client.online_endpoints.begin_create_or_update(endpoint).wait()

# deployment
deployment = KubernetesOnlineDeployment(
    name="iris-endpoint-deployment",
    endpoint_name="iris-endpoint",
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="./scoring_script",
        scoring_script="scoring_script.py"
    ),
    instance_count=1
)
ml_client.online_deployments.begin_create_or_update(deployment).wait()
